In [1]:
from qiskit_nature.units import *   #ANGSTROM
from qiskit_nature.second_q.drivers import *    #PySCFDriver
from qiskit_nature.second_q.mappers import * #JordanWignerMapper
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.problems import EigenstateResult
from qiskit_nature.second_q.circuit.library import *  #Ansatz, HF
#from qiskit_ibm_runtime import QiskitRuntimeService, Session, Estimator, Options
from qiskit_algorithms.optimizers import * 
from qiskit_algorithms import VQE
from braket.tracking import Tracker
from qiskit_braket_provider import *
from qiskit_aer import StatevectorSimulator
from qiskit.primitives import Estimator
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer

In [2]:
driver = PySCFDriver(atom="O 0 0 -1.16; C 0 0 0; O 0 0 1.16", basis="sto-3g")
estrutura_inicial = driver.run() #Criamos a estrutura do nosso sistema

espaco_target_mid = ActiveSpaceTransformer(8,8) #Active Space para o H2
espaco_target_minimal = ActiveSpaceTransformer(4,4) #Active Space para o H2

estrutura_mid = espaco_target_mid.transform(estrutura_inicial) #mudando nossa hamiltoniana para o active space
estrutura_minimal = espaco_target_minimal.transform(estrutura_inicial)


In [3]:
op_i = estrutura_inicial.hamiltonian.second_q_op()

op_mid = estrutura_mid.hamiltonian.second_q_op()
op_minimal = estrutura_minimal.hamiltonian.second_q_op()

print('Número de operadores Fermionicos (Estrutura Inicial): ', len(op_i))

print('Número de operadores Fermionicos (Estrutura Ativa Média): ', len(op_mid))
print('Número de operadores Fermionicos (Estrutura Ativa Minima): ', len(op_minimal))

Número de operadores Fermionicos (Estrutura Inicial):  44898
Número de operadores Fermionicos (Estrutura Ativa Média):  3620
Número de operadores Fermionicos (Estrutura Ativa Minima):  456


In [4]:
#Optendo as simetrias que possuem no sistema do CO2, para criar o mapa:

mapa_mid = estrutura_mid.get_tapered_mapper(BravyiKitaevMapper())
mapa_minimal = estrutura_minimal.get_tapered_mapper(BravyiKitaevMapper())

obs_tapered_mid = mapa_mid.map(op_mid)
obs_tapered_minimal = mapa_minimal.map(op_minimal)

In [5]:
print(obs_tapered_mid)
print()
print('Profundidade de circuito ', len(obs_tapered_mid))

#note que são 12 qubits, ou seja diminuimos o requisito computacional.

SparsePauliOp(['IIIIIIIIIIII', 'ZZIIIIZZZIII', 'IIIIIIIIXYYI', 'ZZIIIIZZYXYI', 'IIIIIIIIYIYI', 'ZZIIIIZZYZYI', 'IZIZIZZIZIZI', 'ZZIIIIZZZIXI', 'ZIIZIZIIZIXI', 'IIIIIIIIIIIZ', 'ZIIZIZIZIIIX', 'IIIIIIIIIIIX', 'ZIIZIZIZIIIZ', 'IIIIIIIIIZII', 'IIIIIIIIZXII', 'IIIIIIIIIXII', 'IIIIIIIIZZII', 'IIIIIIIZIIII', 'IIIIIIIZZIZI', 'IIIIIIZIIIII', 'IXYYIXXIIIII', 'IXXYIXYIIIII', 'IYIYIXXIIIII', 'IXZYIXYIIIII', 'IIIIIZZIIIII', 'IIIXIXZIIIII', 'ZZIYIYIIIIII', 'IIIIZIIIIIII', 'IIIZXZIIIIII', 'IIIIXIIIIIII', 'IIIZZZIIIIII', 'IIZIIIIIIIII', 'IZXIIIIIIIII', 'IIXIIIIIIIII', 'IZZIIIIIIIII', 'ZIIIIIIIIIII', 'ZZIZIIIIIIII', 'ZIIZIZIZIIZI', 'IIIIIIIIIIXI', 'IZIZIZZZIIXI', 'ZZIIIIZZZIIZ', 'IZIZIZZIZIIX', 'ZZIIIIZZZIIX', 'IZIZIZZIZIIZ', 'ZZIIIIZZZZII', 'ZZIIIIZZIXII', 'ZZIIIIZZZXII', 'ZZIIIIZZIZII', 'ZZIIIIZIZIII', 'ZZIIIIZIIIZI', 'ZZIIIIIZZIII', 'ZYYYIXYZZIII', 'ZYXYIXXZZIII', 'ZXIYIXYZZIII', 'ZYZYIXXZZIII', 'ZZIIIZIZZIII', 'ZZIXIXIZZIII', 'IIIYIYZZZIII', 'ZZIIZIZZZIII', 'ZZIZXZZZZIII', 'ZZIIXIZZZIII', 'ZZIZZZZZ

In [6]:
print(obs_tapered_minimal)
print()
print('Profundidade de circuito ', len(obs_tapered_minimal))

#note que são 5 qubits, ou seja diminuimos o requisito computacional.

SparsePauliOp(['IIIII', 'IIIIZ', 'IZIIZ', 'IIIZI', 'IZIZI', 'IIZII', 'IZZII', 'ZIIII', 'ZZIII', 'IZIII', 'IIIZZ', 'IZIXZ', 'IIIXZ', 'IZIZZ', 'IIZIZ', 'IZZIZ', 'ZIIIZ', 'XZIIZ', 'XIIIZ', 'ZZIIZ', 'IZIZX', 'IIIZX', 'IIIXX', 'IZIXX', 'IZIYY', 'IIXIX', 'IZXIX', 'ZZIIX', 'ZIIIX', 'XIIIX', 'XZIIX', 'YXXYX', 'XXYYX', 'YXXXY', 'XXYXY', 'IYXYX', 'ZXYYX', 'IYXXY', 'ZXYXY', 'YXZYX', 'XYIYX', 'YXZXY', 'XYIXY', 'IYZYX', 'ZYIYX', 'IYZXY', 'ZYIXY', 'YYXIX', 'XYYIX', 'YXXZY', 'XXYZY', 'IXXIX', 'ZYYIX', 'IYXZY', 'ZXYZY', 'YYZIX', 'XXIIX', 'YXZZY', 'XYIZY', 'IXZIX', 'ZXIIX', 'IYZZY', 'ZYIZY', 'YXXYZ', 'XXYYZ', 'YYXXI', 'XYYXI', 'IYXYZ', 'ZXYYZ', 'IXXXI', 'ZYYXI', 'YXZYZ', 'XYIYZ', 'YYZXI', 'XXIXI', 'IYZYZ', 'ZYIYZ', 'IXZXI', 'ZXIXI', 'YYXIZ', 'XYYIZ', 'YYXZI', 'XYYZI', 'IXXIZ', 'ZYYIZ', 'IXXZI', 'ZYYZI', 'YYZIZ', 'XXIIZ', 'YYZZI', 'XXIZI', 'IXZIZ', 'ZXIIZ', 'IXZZI', 'ZXIZI', 'IIZZI', 'IZXZI', 'IIXZI', 'IZZZI', 'ZIIZI', 'ZZIZI', 'IZZXI', 'IIZXI', 'IIXXI', 'IZXXI', 'XIIXI', 'XZIXI', 'ZIZII', 'XZZII', 'XIZ

In [56]:
ansatz = UCCSD(
    estrutura_minimal.num_spatial_orbitals,
    estrutura_minimal.num_particles,
    mapa_minimal,
    initial_state=HartreeFock(
        estrutura_minimal.num_spatial_orbitals,
        estrutura_minimal.num_particles,
        mapa_minimal))


vqe_solver = VQE(Estimator(), ansatz, SLSQP())
vqe_solver.initial_point = [0.0] * ansatz.num_parameters
calculo = GroundStateEigensolver(mapa_minimal, vqe_solver)
resultados = calculo.solve(estrutura_minimal)

In [60]:
print(resultados)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -243.529028939878
  - computed part:      -4.722698130617
  - ActiveSpaceTransformer extracted energy part: -238.806330809261
~ Nuclear repulsion energy (Hartree): 58.391968101517
> Total ground state energy (Hartree): -185.137060838361
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  None]
    - computed part:      [0.0  0.0  None]
    - ActiveSpaceTransformer extracted energy part: [0.0  0.0  -0.000000000001]
  > Dipole moment (a.u.): [0.0  0.0  None]  Total: None
                 (debye): [0.0  0.0  None]  Total: None
 
